In [1]:
import os,random,math
TRAINING_DIR="propaganda_dataset_v2"
def get_propoganda_train(training_dir=TRAINING_DIR):

    filenames=os.listdir(training_dir)
    n=len(filenames)
    print("There are {} files in the training directory: {}".format(n,training_dir))
    return filenames

In [2]:
filename=get_propoganda_train()

There are 2 files in the training directory: propaganda_dataset_v2


In [3]:
import pandas as pd
def read_tsvfiles(filename):
    tsv_filename= os.path.join(TRAINING_DIR,filename[0])
    df = pd.read_csv(tsv_filename, sep='\t')
    return df  

In [4]:
import pandas as pd
def read_tsvtestfiles(filename):
    tsv_filename= os.path.join(TRAINING_DIR,filename[1])
    df = pd.read_csv(tsv_filename, sep='\t')
    return df  

In [5]:
df= read_tsvfiles(filename)
ts = os.path.join(TRAINING_DIR,filename[0])
df

,label,tagged_in_context
0,not_propaganda,"No, <BOS> he <EOS> will not be confirmed."
1,not_propaganda,This declassification effort <BOS> won’t make ...
2,flag_waving,The Obama administration misled the <BOS> Amer...
3,not_propaganda,“It looks like we’re capturing the demise of t...
4,not_propaganda,"<BOS> Location: Westerville, Ohio <EOS>"
...,...,...
2409,not_propaganda,<BOS> We support and appreciate <EOS> your bus...
2410,not_propaganda,International Atomic Energy Agency (IAEA) Dire...
2411,not_propaganda,What has been done: there has been work on for...
2412,not_propaganda,This is <BOS> the law of gradualness not the g...


In [6]:
test_df= read_tsvtestfiles(filename)
ts_df = os.path.join(TRAINING_DIR,filename[1])
test_df

,label,tagged_in_context
0,not_propaganda,"On average, between 300 and 600 infections are..."
1,causal_oversimplification,Mostly because <BOS> the country would not las...
2,appeal_to_fear_prejudice,Lyndon Johnson <BOS> gets Earl Warren and Sen....
3,not_propaganda,<BOS> You <EOS> may opt out at anytime.
4,repetition,It must be exacted from him directly in order ...
...,...,...
575,not_propaganda,"NewsCatholic Church, <BOS> Family, Marriage <E..."
576,not_propaganda,"Remember our saying, modern day fairy <BOS> ta..."
577,not_propaganda,Why <BOS> not <EOS> open up to Iran with massi...
578,flag_waving,<BOS> He also sang an Islamic State fight song...


In [7]:
df["tagged_in_context"][3]

'“It looks like we’re capturing the demise of this dark vortex, <BOS> and <EOS> it’s different from what well-known studies led us to expect,” said Michael H. Wong of the University of California at Berkeley says. '

In [8]:
import numpy as np
import tensorflow as tf
import keras as ke

In [9]:
%pip install --upgrade gensim

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [11]:
from gensim.models.doc2vec import Word2Vec, TaggedDocument
import numpy as np
import tensorflow_hub as hub
from nltk.tokenize import word_tokenize

In [12]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [13]:
def convert_token(sent):
    text_array = sent.split("<BOS>")
    stat = text_array[1].split("<EOS>")
    #stat_token = word_tokenize(stat[0])
    return stat[0]

In [14]:
df['stat_token']= df["tagged_in_context"].map(convert_token)

In [15]:
test_df['stat_token']= test_df["tagged_in_context"].map(convert_token)

In [16]:
df

,label,tagged_in_context,stat_token
0,not_propaganda,"No, <BOS> he <EOS> will not be confirmed.",he
1,not_propaganda,This declassification effort <BOS> won’t make ...,won’t make things any worse than they are for...
2,flag_waving,The Obama administration misled the <BOS> Amer...,American people
3,not_propaganda,“It looks like we’re capturing the demise of t...,and
4,not_propaganda,"<BOS> Location: Westerville, Ohio <EOS>","Location: Westerville, Ohio"
...,...,...,...
2409,not_propaganda,<BOS> We support and appreciate <EOS> your bus...,We support and appreciate
2410,not_propaganda,International Atomic Energy Agency (IAEA) Dire...,capacity to check whether Iran was conducting...
2411,not_propaganda,What has been done: there has been work on for...,one for those recently
2412,not_propaganda,This is <BOS> the law of gradualness not the g...,the law of gradualness not the gradualness of...


In [17]:
test_df

,label,tagged_in_context,stat_token
0,not_propaganda,"On average, between 300 and 600 infections are...",according to a UN estimate.
1,causal_oversimplification,Mostly because <BOS> the country would not las...,the country would not last long without an ou...
2,appeal_to_fear_prejudice,Lyndon Johnson <BOS> gets Earl Warren and Sen....,gets Earl Warren and Sen. Richard Russel to j...
3,not_propaganda,<BOS> You <EOS> may opt out at anytime.,You
4,repetition,It must be exacted from him directly in order ...,infidels
...,...,...,...
575,not_propaganda,"NewsCatholic Church, <BOS> Family, Marriage <E...","Family, Marriage"
576,not_propaganda,"Remember our saying, modern day fairy <BOS> ta...",tales start with ‘once I am
577,not_propaganda,Why <BOS> not <EOS> open up to Iran with massi...,not
578,flag_waving,<BOS> He also sang an Islamic State fight song...,He also sang an Islamic State fight song and ...


In [18]:
from nltk import word_tokenize
df['tokens']=df['stat_token'].apply(word_tokenize)
df

,label,tagged_in_context,stat_token,tokens
0,not_propaganda,"No, <BOS> he <EOS> will not be confirmed.",he,[he]
1,not_propaganda,This declassification effort <BOS> won’t make ...,won’t make things any worse than they are for...,"[won, ’, t, make, things, any, worse, than, th..."
2,flag_waving,The Obama administration misled the <BOS> Amer...,American people,"[American, people]"
3,not_propaganda,“It looks like we’re capturing the demise of t...,and,[and]
4,not_propaganda,"<BOS> Location: Westerville, Ohio <EOS>","Location: Westerville, Ohio","[Location, :, Westerville, ,, Ohio]"
...,...,...,...,...
2409,not_propaganda,<BOS> We support and appreciate <EOS> your bus...,We support and appreciate,"[We, support, and, appreciate]"
2410,not_propaganda,International Atomic Energy Agency (IAEA) Dire...,capacity to check whether Iran was conducting...,"[capacity, to, check, whether, Iran, was, cond..."
2411,not_propaganda,What has been done: there has been work on for...,one for those recently,"[one, for, those, recently]"
2412,not_propaganda,This is <BOS> the law of gradualness not the g...,the law of gradualness not the gradualness of...,"[the, law, of, gradualness, not, the, gradualn..."


In [19]:
test_df['tokens']=test_df['stat_token'].apply(word_tokenize)
test_df

,label,tagged_in_context,stat_token,tokens
0,not_propaganda,"On average, between 300 and 600 infections are...",according to a UN estimate.,"[according, to, a, UN, estimate, .]"
1,causal_oversimplification,Mostly because <BOS> the country would not las...,the country would not last long without an ou...,"[the, country, would, not, last, long, without..."
2,appeal_to_fear_prejudice,Lyndon Johnson <BOS> gets Earl Warren and Sen....,gets Earl Warren and Sen. Richard Russel to j...,"[gets, Earl, Warren, and, Sen., Richard, Russe..."
3,not_propaganda,<BOS> You <EOS> may opt out at anytime.,You,[You]
4,repetition,It must be exacted from him directly in order ...,infidels,[infidels]
...,...,...,...,...
575,not_propaganda,"NewsCatholic Church, <BOS> Family, Marriage <E...","Family, Marriage","[Family, ,, Marriage]"
576,not_propaganda,"Remember our saying, modern day fairy <BOS> ta...",tales start with ‘once I am,"[tales, start, with, ‘, once, I, am]"
577,not_propaganda,Why <BOS> not <EOS> open up to Iran with massi...,not,[not]
578,flag_waving,<BOS> He also sang an Islamic State fight song...,He also sang an Islamic State fight song and ...,"[He, also, sang, an, Islamic, State, fight, so..."


In [20]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

def normalise(wordlist):
    lowered=[word.lower() for word in wordlist] #don't actually need this as already lowered
    filtered=[word for word in lowered if word.isalpha()]
    return filtered


In [21]:
df['filtered-token']=df['tokens'].map(normalise)
df

,label,tagged_in_context,stat_token,tokens,filtered-token
0,not_propaganda,"No, <BOS> he <EOS> will not be confirmed.",he,[he],[he]
1,not_propaganda,This declassification effort <BOS> won’t make ...,won’t make things any worse than they are for...,"[won, ’, t, make, things, any, worse, than, th...","[won, t, make, things, any, worse, than, they,..."
2,flag_waving,The Obama administration misled the <BOS> Amer...,American people,"[American, people]","[american, people]"
3,not_propaganda,“It looks like we’re capturing the demise of t...,and,[and],[and]
4,not_propaganda,"<BOS> Location: Westerville, Ohio <EOS>","Location: Westerville, Ohio","[Location, :, Westerville, ,, Ohio]","[location, westerville, ohio]"
...,...,...,...,...,...
2409,not_propaganda,<BOS> We support and appreciate <EOS> your bus...,We support and appreciate,"[We, support, and, appreciate]","[we, support, and, appreciate]"
2410,not_propaganda,International Atomic Energy Agency (IAEA) Dire...,capacity to check whether Iran was conducting...,"[capacity, to, check, whether, Iran, was, cond...","[capacity, to, check, whether, iran, was, cond..."
2411,not_propaganda,What has been done: there has been work on for...,one for those recently,"[one, for, those, recently]","[one, for, those, recently]"
2412,not_propaganda,This is <BOS> the law of gradualness not the g...,the law of gradualness not the gradualness of...,"[the, law, of, gradualness, not, the, gradualn...","[the, law, of, gradualness, not, the, gradualn..."


In [22]:
test_df['filtered-token']=test_df['tokens'].map(normalise)
test_df

,label,tagged_in_context,stat_token,tokens,filtered-token
0,not_propaganda,"On average, between 300 and 600 infections are...",according to a UN estimate.,"[according, to, a, UN, estimate, .]","[according, to, a, un, estimate]"
1,causal_oversimplification,Mostly because <BOS> the country would not las...,the country would not last long without an ou...,"[the, country, would, not, last, long, without...","[the, country, would, not, last, long, without..."
2,appeal_to_fear_prejudice,Lyndon Johnson <BOS> gets Earl Warren and Sen....,gets Earl Warren and Sen. Richard Russel to j...,"[gets, Earl, Warren, and, Sen., Richard, Russe...","[gets, earl, warren, and, richard, russel, to,..."
3,not_propaganda,<BOS> You <EOS> may opt out at anytime.,You,[You],[you]
4,repetition,It must be exacted from him directly in order ...,infidels,[infidels],[infidels]
...,...,...,...,...,...
575,not_propaganda,"NewsCatholic Church, <BOS> Family, Marriage <E...","Family, Marriage","[Family, ,, Marriage]","[family, marriage]"
576,not_propaganda,"Remember our saying, modern day fairy <BOS> ta...",tales start with ‘once I am,"[tales, start, with, ‘, once, I, am]","[tales, start, with, once, i, am]"
577,not_propaganda,Why <BOS> not <EOS> open up to Iran with massi...,not,[not],[not]
578,flag_waving,<BOS> He also sang an Islamic State fight song...,He also sang an Islamic State fight song and ...,"[He, also, sang, an, Islamic, State, fight, so...","[he, also, sang, an, islamic, state, fight, so..."


In [24]:
%pip install Keras-Preprocessing

     ---------------------------------------- 42.6/42.6 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, Dense, CuDNNLSTM, concatenate, Bidirectional, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
from sklearn.model_selection import KFold

In [26]:
vocab_list=list(df['filtered-token'])+list(test_df['filtered-token'])
tokenizer= Tokenizer(num_words=100000)
tokenizer.fit_on_texts(vocab_list)
df

,label,tagged_in_context,stat_token,tokens,filtered-token
0,not_propaganda,"No, <BOS> he <EOS> will not be confirmed.",he,[he],[he]
1,not_propaganda,This declassification effort <BOS> won’t make ...,won’t make things any worse than they are for...,"[won, ’, t, make, things, any, worse, than, th...","[won, t, make, things, any, worse, than, they,..."
2,flag_waving,The Obama administration misled the <BOS> Amer...,American people,"[American, people]","[american, people]"
3,not_propaganda,“It looks like we’re capturing the demise of t...,and,[and],[and]
4,not_propaganda,"<BOS> Location: Westerville, Ohio <EOS>","Location: Westerville, Ohio","[Location, :, Westerville, ,, Ohio]","[location, westerville, ohio]"
...,...,...,...,...,...
2409,not_propaganda,<BOS> We support and appreciate <EOS> your bus...,We support and appreciate,"[We, support, and, appreciate]","[we, support, and, appreciate]"
2410,not_propaganda,International Atomic Energy Agency (IAEA) Dire...,capacity to check whether Iran was conducting...,"[capacity, to, check, whether, Iran, was, cond...","[capacity, to, check, whether, iran, was, cond..."
2411,not_propaganda,What has been done: there has been work on for...,one for those recently,"[one, for, those, recently]","[one, for, those, recently]"
2412,not_propaganda,This is <BOS> the law of gradualness not the g...,the law of gradualness not the gradualness of...,"[the, law, of, gradualness, not, the, gradualn...","[the, law, of, gradualness, not, the, gradualn..."


In [27]:
test_tokenizer= Tokenizer(num_words=100000)
test_tokenizer.fit_on_texts(list(test_df['filtered-token']))
test_df

,label,tagged_in_context,stat_token,tokens,filtered-token
0,not_propaganda,"On average, between 300 and 600 infections are...",according to a UN estimate.,"[according, to, a, UN, estimate, .]","[according, to, a, un, estimate]"
1,causal_oversimplification,Mostly because <BOS> the country would not las...,the country would not last long without an ou...,"[the, country, would, not, last, long, without...","[the, country, would, not, last, long, without..."
2,appeal_to_fear_prejudice,Lyndon Johnson <BOS> gets Earl Warren and Sen....,gets Earl Warren and Sen. Richard Russel to j...,"[gets, Earl, Warren, and, Sen., Richard, Russe...","[gets, earl, warren, and, richard, russel, to,..."
3,not_propaganda,<BOS> You <EOS> may opt out at anytime.,You,[You],[you]
4,repetition,It must be exacted from him directly in order ...,infidels,[infidels],[infidels]
...,...,...,...,...,...
575,not_propaganda,"NewsCatholic Church, <BOS> Family, Marriage <E...","Family, Marriage","[Family, ,, Marriage]","[family, marriage]"
576,not_propaganda,"Remember our saying, modern day fairy <BOS> ta...",tales start with ‘once I am,"[tales, start, with, ‘, once, I, am]","[tales, start, with, once, i, am]"
577,not_propaganda,Why <BOS> not <EOS> open up to Iran with massi...,not,[not],[not]
578,flag_waving,<BOS> He also sang an Islamic State fight song...,He also sang an Islamic State fight song and ...,"[He, also, sang, an, Islamic, State, fight, so...","[he, also, sang, an, islamic, state, fight, so..."


In [28]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'in': 6,
 'that': 7,
 'is': 8,
 'it': 9,
 'for': 10,
 's': 11,
 'not': 12,
 'with': 13,
 'on': 14,
 'they': 15,
 'are': 16,
 'be': 17,
 'this': 18,
 'he': 19,
 'was': 20,
 'as': 21,
 'have': 22,
 'by': 23,
 'from': 24,
 'his': 25,
 'who': 26,
 'an': 27,
 'our': 28,
 'we': 29,
 'has': 30,
 'will': 31,
 'you': 32,
 'do': 33,
 'people': 34,
 'at': 35,
 'all': 36,
 'if': 37,
 'i': 38,
 'church': 39,
 'one': 40,
 'their': 41,
 'or': 42,
 'but': 43,
 'about': 44,
 't': 45,
 'can': 46,
 'what': 47,
 'american': 48,
 'out': 49,
 'trump': 50,
 'more': 51,
 'them': 52,
 'would': 53,
 'which': 54,
 'been': 55,
 'no': 56,
 'any': 57,
 'up': 58,
 'so': 59,
 'against': 60,
 'were': 61,
 'america': 62,
 'because': 63,
 'had': 64,
 'its': 65,
 'said': 66,
 'should': 67,
 'country': 68,
 'over': 69,
 'there': 70,
 'world': 71,
 'when': 72,
 'those': 73,
 'never': 74,
 'only': 75,
 'her': 76,
 'him': 77,
 'god': 78,
 'most': 79,
 'could': 80,
 'many': 

In [29]:
len(tokenizer.word_index)

5177

In [30]:
test_tokenizer.word_index

{'the': 1,
 'to': 2,
 'of': 3,
 'and': 4,
 'a': 5,
 'in': 6,
 'that': 7,
 'is': 8,
 'it': 9,
 'with': 10,
 's': 11,
 'for': 12,
 'not': 13,
 'on': 14,
 'be': 15,
 'as': 16,
 'has': 17,
 'he': 18,
 'this': 19,
 'are': 20,
 'they': 21,
 'was': 22,
 'we': 23,
 'his': 24,
 'by': 25,
 'have': 26,
 'will': 27,
 'do': 28,
 'our': 29,
 'you': 30,
 'at': 31,
 'but': 32,
 'or': 33,
 'who': 34,
 'all': 35,
 'people': 36,
 'there': 37,
 'what': 38,
 'up': 39,
 'if': 40,
 'an': 41,
 'about': 42,
 'from': 43,
 'their': 44,
 'would': 45,
 'its': 46,
 'been': 47,
 'them': 48,
 'america': 49,
 'did': 50,
 'one': 51,
 'which': 52,
 'had': 53,
 'more': 54,
 'out': 55,
 'no': 56,
 'because': 57,
 'any': 58,
 'american': 59,
 't': 60,
 'can': 61,
 'him': 62,
 'trump': 63,
 'world': 64,
 'my': 65,
 'said': 66,
 'when': 67,
 'public': 68,
 'over': 69,
 'country': 70,
 'without': 71,
 'against': 72,
 'now': 73,
 'i': 74,
 'only': 75,
 'so': 76,
 'never': 77,
 'make': 78,
 'right': 79,
 'sexual': 80,
 'god': 8

In [31]:
sequences = tokenizer.texts_to_sequences(list(df['filtered-token']))
test_sequences = tokenizer.texts_to_sequences(list(test_df['filtered-token']))
sequences
#test_sequences

[[19],
 [381, 45, 95, 382, 57, 186, 88, 15, 16, 10, 104, 50],
 [48, 34],
 [4],
 [2338, 2339, 2340],
 [1011],
 [2,
  1428,
  7,
  1429,
  1430,
  6,
  7,
  68,
  53,
  95,
  9,
  522,
  2,
  523,
  213,
  158,
  2341,
  291,
  69,
  292,
  524,
  2,
  1,
  2342],
 [2343, 1431, 2344],
 [148],
 [791, 792],
 [1432, 1433],
 [293, 2, 444, 2],
 [294,
  2,
  119,
  11,
  2345,
  445,
  10,
  51,
  793,
  383,
  384,
  7,
  80,
  295,
  385,
  6,
  794,
  11,
  170],
 [214, 795, 1012, 6, 1, 1],
 [51],
 [1434, 3, 2346, 386, 5, 330, 796, 23, 113, 132],
 [12],
 [2347, 59],
 [43, 26, 1435, 18, 2348, 628],
 [1,
  75,
  331,
  114,
  70,
  8,
  263,
  2349,
  6,
  1,
  1436,
  27,
  525,
  8,
  2,
  1013,
  27,
  797,
  2,
  1,
  387,
  3,
  526,
  42,
  12,
  9,
  2350,
  4,
  2351,
  159,
  527],
 [29, 120, 332, 1014, 114, 18, 1015, 45, 333],
 [2352],
 [62],
 [8, 1, 294, 44, 1, 264, 528, 7, 89],
 [446],
 [1437, 1, 529, 30],
 [5, 296, 2, 388],
 [2353, 447],
 [],
 [1438, 4, 5, 1439, 3],
 [1, 179, 7, 

In [32]:
from keras.utils import pad_sequences

In [33]:
sequences=pad_sequences(sequences,padding='post',maxlen=140)
test_sequences=pad_sequences(test_sequences,padding='post',maxlen=140)
len(test_sequences[0])

140

In [34]:
def labels(lab):
    label=0
    if lab=='not_propaganda':
        label=1
    else:
        label=0
    return label

df['class_label']=df['label'].map(labels)
test_df['class_label']=test_df['label'].map(labels)
test_df

,label,tagged_in_context,stat_token,tokens,filtered-token,class_label
0,not_propaganda,"On average, between 300 and 600 infections are...",according to a UN estimate.,"[according, to, a, UN, estimate, .]","[according, to, a, un, estimate]",1
1,causal_oversimplification,Mostly because <BOS> the country would not las...,the country would not last long without an ou...,"[the, country, would, not, last, long, without...","[the, country, would, not, last, long, without...",0
2,appeal_to_fear_prejudice,Lyndon Johnson <BOS> gets Earl Warren and Sen....,gets Earl Warren and Sen. Richard Russel to j...,"[gets, Earl, Warren, and, Sen., Richard, Russe...","[gets, earl, warren, and, richard, russel, to,...",0
3,not_propaganda,<BOS> You <EOS> may opt out at anytime.,You,[You],[you],1
4,repetition,It must be exacted from him directly in order ...,infidels,[infidels],[infidels],0
...,...,...,...,...,...,...
575,not_propaganda,"NewsCatholic Church, <BOS> Family, Marriage <E...","Family, Marriage","[Family, ,, Marriage]","[family, marriage]",1
576,not_propaganda,"Remember our saying, modern day fairy <BOS> ta...",tales start with ‘once I am,"[tales, start, with, ‘, once, I, am]","[tales, start, with, once, i, am]",1
577,not_propaganda,Why <BOS> not <EOS> open up to Iran with massi...,not,[not],[not],1
578,flag_waving,<BOS> He also sang an Islamic State fight song...,He also sang an Islamic State fight song and ...,"[He, also, sang, an, Islamic, State, fight, so...","[he, also, sang, an, islamic, state, fight, so...",0


In [35]:
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding,Flatten

In [36]:
def length(tok):
    le=len(tok)
    return le
test_df['length']=test_df['filtered-token'].map(length)
test_df

,label,tagged_in_context,stat_token,tokens,filtered-token,class_label,length
0,not_propaganda,"On average, between 300 and 600 infections are...",according to a UN estimate.,"[according, to, a, UN, estimate, .]","[according, to, a, un, estimate]",1,5
1,causal_oversimplification,Mostly because <BOS> the country would not las...,the country would not last long without an ou...,"[the, country, would, not, last, long, without...","[the, country, would, not, last, long, without...",0,16
2,appeal_to_fear_prejudice,Lyndon Johnson <BOS> gets Earl Warren and Sen....,gets Earl Warren and Sen. Richard Russel to j...,"[gets, Earl, Warren, and, Sen., Richard, Russe...","[gets, earl, warren, and, richard, russel, to,...",0,23
3,not_propaganda,<BOS> You <EOS> may opt out at anytime.,You,[You],[you],1,1
4,repetition,It must be exacted from him directly in order ...,infidels,[infidels],[infidels],0,1
...,...,...,...,...,...,...,...
575,not_propaganda,"NewsCatholic Church, <BOS> Family, Marriage <E...","Family, Marriage","[Family, ,, Marriage]","[family, marriage]",1,2
576,not_propaganda,"Remember our saying, modern day fairy <BOS> ta...",tales start with ‘once I am,"[tales, start, with, ‘, once, I, am]","[tales, start, with, once, i, am]",1,6
577,not_propaganda,Why <BOS> not <EOS> open up to Iran with massi...,not,[not],[not],1,1
578,flag_waving,<BOS> He also sang an Islamic State fight song...,He also sang an Islamic State fight song and ...,"[He, also, sang, an, Islamic, State, fight, so...","[he, also, sang, an, islamic, state, fight, so...",0,31


In [37]:
ma=max(test_df['length'])
ma

72

In [38]:
model = Sequential()
model.add(Embedding(5179,2,input_length=140))
model.add(SimpleRNN(32,return_sequences=False))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 140, 2)            10358     
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1120      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 11,511
Trainable params: 11,511
Non-trainable params: 0
_________________________________________________________________


In [39]:
train_y= df['class_label'].to_numpy()
type(train_y)

numpy.ndarray

In [40]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
train_y=np.array(df['class_label'])
test_y=np.array(test_df['class_label'])
history = model.fit(sequences,train_y, epochs=10,validation_data=(test_sequences,test_y) )

Epoch 1/10
76/76 [==============================] - 6s 43ms/step - loss: 0.6955 - accuracy: 0.5046 - val_loss: 0.6969 - val_accuracy: 0.4810
Epoch 2/10
76/76 [==============================] - 3s 40ms/step - loss: 0.6946 - accuracy: 0.5004 - val_loss: 0.7037 - val_accuracy: 0.4690
Epoch 3/10
76/76 [==============================] - 3s 38ms/step - loss: 0.6975 - accuracy: 0.4925 - val_loss: 0.6935 - val_accuracy: 0.4948
Epoch 4/10
76/76 [==============================] - 3s 37ms/step - loss: 0.6953 - accuracy: 0.5058 - val_loss: 0.6889 - val_accuracy: 0.5586
Epoch 5/10
76/76 [==============================] - 3s 43ms/step - loss: 0.6928 - accuracy: 0.5220 - val_loss: 0.6889 - val_accuracy: 0.5466
Epoch 6/10
76/76 [==============================] - 3s 42ms/step - loss: 0.6714 - accuracy: 0.6002 - val_loss: 0.6943 - val_accuracy: 0.5414
Epoch 7/10
76/76 [==============================] - 3s 38ms/step - loss: 0.6491 - accuracy: 0.6218 - val_loss: 0.6994 - val_accuracy: 0.5397
Epoch 8/10
76

In [41]:
train_y=np.array(df['class_label'])
train_y

array([1, 1, 0, ..., 1, 1, 0], dtype=int64)

In [42]:
sequences

array([[  19,    0,    0, ...,    0,    0,    0],
       [ 381,   45,   95, ...,    0,    0,    0],
       [  48,   34,    0, ...,    0,    0,    0],
       ...,
       [  40,   10,   73, ...,    0,    0,    0],
       [   1,  158,    3, ...,    0,    0,    0],
       [4641,    0,    0, ...,    0,    0,    0]])

In [44]:
from sklearn.metrics import confusion_matrix
y_prediction = model.predict(test_sequences)
y_prediction = np.argmax (y_prediction, axis = 1)
y_test=np.argmax(test_y, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_prediction , normalize='pred')
print(result)

19/19 [==============================] - 0s 10ms/step


AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
y_prediction = model.predict(sequences)
print(y_prediction)

76/76 [==============================] - 2s 21ms/step
[[0.949272  ]
 [0.8449858 ]
 [0.01171791]
 ...
 [0.90020007]
 [0.9127499 ]
 [0.01684237]]
